### Setting Up

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/ECE1786_Project/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/ECE1786_Project


In [2]:
!pip install wandb

import wandb
wandb.login()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dhairya18498. Use `wandb login --relogin` to force relogin


True

In [3]:
!pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Importing Data & Creating Splits

In [5]:
import pandas as pd
dhairya_df = pd.read_csv('Dataset_Dhairya.csv')
yuchen_df = pd.read_csv('Dataset_Yuchen.csv')

In [6]:
dhairya_df.head()

,text,code
0,Write a python program for time Complexity pl...,import time\nfrom numpy.random import seed\nfr...
1,Write a python function for Stooge Sort\n,"def stoogesort(arr, l, h):\n if l >= h:\n ret..."
2,Write a python function for Recursive Inserti...,"def insertionSortRecursive(arr, n):\n if n ..."
3,Write a python function for Cycle Sort\n,def cycleSort(array):\n writes = 0\n \n fo...
4,Write a python program for BogoSort\n,import random\n\ndef is_sorted(a):\n n = le...


In [7]:
yuchen_df.head()

,text,code
0,write a python program to sort an array in asc...,def bubbleSort(array):\n swapped = False\n ...
1,write a python program to sort an array in des...,def bubbleSort(array):\n swapped = False\n ...
2,write a python program to find the element tha...,def bubbleSort(array):\n swapped = False\n ...
3,write a python program to check if any value a...,def bubbleSort(array):\n swapped = False\n ...
4,write a python program to find the third disti...,def bubbleSort(array):\n swapped = False\n ...


In [8]:
finetuning_dataset = pd.concat([dhairya_df, yuchen_df], ignore_index = True)
finetuning_dataset.head()

,text,code
0,Write a python program for time Complexity pl...,import time\nfrom numpy.random import seed\nfr...
1,Write a python function for Stooge Sort\n,"def stoogesort(arr, l, h):\n if l >= h:\n ret..."
2,Write a python function for Recursive Inserti...,"def insertionSortRecursive(arr, n):\n if n ..."
3,Write a python function for Cycle Sort\n,def cycleSort(array):\n writes = 0\n \n fo...
4,Write a python program for BogoSort\n,import random\n\ndef is_sorted(a):\n n = le...


In [9]:
from sklearn.model_selection import train_test_split
finetuning_train_val,  finetuning_test = train_test_split(finetuning_dataset, test_size = 0.10, random_state = 42)
finetuning_train,  finetuning_val = train_test_split(finetuning_train_val, test_size = 0.10, random_state = 42)

In [10]:
finetuning_val.head()

,text,code
14,Write a python program for Binary Insertion S...,"def binary_search(arr, val, start, end):\n ..."
34,Write a Python program to sort a given collec...,"def rec_insertion_sort(collection: list, n: in..."
50,Write a python program for sorting a list usi...,def cocktailSort(a):\n n = len(a)\n swap...
4,Write a python program for BogoSort\n,import random\n\ndef is_sorted(a):\n n = le...
141,write a python program to find out how many nu...,def mergeSort(nums):\n if len(nums)==1:\n ...


In [11]:
from datasets import Dataset
training_set = Dataset.from_pandas(finetuning_train)
validation_set = Dataset.from_pandas(finetuning_val)
test_set = Dataset.from_pandas(finetuning_test)

In [12]:
training_set = training_set.remove_columns([list(training_set.features.keys())[-1]])
training_set

Dataset({
    features: ['text', 'code'],
    num_rows: 145
})

In [13]:
validation_set = validation_set.remove_columns([list(validation_set.features.keys())[-1]])
validation_set

Dataset({
    features: ['text', 'code'],
    num_rows: 17
})

### Model Definition

In [14]:
from transformers import AutoTokenizer, AutoModelWithLMHead

tokenizer = AutoTokenizer.from_pretrained("Salesforce/codet5-base")

model = AutoModelWithLMHead.from_pretrained("Salesforce/codet5-base")

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:1136: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


### Pre-Processing

In [15]:
def prepare_data(examples):
    text = examples['text']
    code = examples['code']
    
    model_inputs = tokenizer(text, padding="longest", truncation = True)
    labels = tokenizer(code, padding="longest", truncation = True).input_ids

    labels_with_ignore_index = []
    for labels_example in labels:
      labels_example = [label if label != 0 else -100 for label in labels_example]
      labels_with_ignore_index.append(labels_example)
    
    model_inputs["labels"] = labels_with_ignore_index

    return model_inputs

In [16]:
tokenized_training_set = training_set.map(prepare_data, batched=True, remove_columns=training_set.column_names)
tokenized_validation_set = validation_set.map(prepare_data, batched=True, remove_columns=validation_set.column_names)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [17]:
labels = tokenized_training_set["labels"][0]
print(tokenizer.decode([label for label in labels if label != -100]))

<s>import itertools
def radixSort(array):
    min_num = min(array)
    if min_num < 0:
        lb = abs(min_num)
        array = [item + lb for item in array]
    n_digits = len(str(max(array)))
    for dgt in range(n_digits):
        buckets = [[] for i in range(10)]
        for num in array:
            idx = (num // (10**dgt)) % 10
            buckets[idx].append(num)
        array = list(itertools.chain(*buckets))
    if min_num < 0:
        lb = abs(min_num)
        array = [item - lb for item in array]
    return array
arr = [1,2,3,4]
arr = radixSort(arr)
print(max(arr[-1] * arr[-2] * arr[-3], arr[0] * arr[1] * arr[-1]))




</s>


### Training & Saving

In [18]:
from transformers import TrainingArguments

args = TrainingArguments(
    "/content/drive/MyDrive/ECE1786_Project/results",
    logging_steps = 36,
    evaluation_strategy = "epoch",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=20,
    learning_rate=2e-5,
    weight_decay=0.01,
    report_to="wandb"
)

In [19]:
from transformers import default_data_collator

data_collator = default_data_collator

In [20]:
import torch, gc
torch.cuda.empty_cache()
gc.collect()

85

In [21]:
from transformers import Trainer

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_training_set,
    eval_dataset=tokenized_validation_set,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [22]:
import torch, gc
torch.cuda.empty_cache()
gc.collect()

22

In [23]:
wandb.init(project="codet5_base_finetuning", entity="dhairya18498")

In [24]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 145
  Num Epochs = 20
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 1460
  Number of trainable parameters = 222882048
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss
1,2.502300,2.191285
2,1.611700,1.646438
3,1.216000,1.462617
4,1.061400,1.320475
5,1.002400,1.232789
6,0.871700,1.161271
7,0.724200,1.122418
8,0.673500,1.055563
9,0.605100,1.020484
10,0.536100,0.984825


***** Running Evaluation *****
  Num examples = 17
  Batch size = 2
***** Running Evaluation *****
  Num examples = 17
  Batch size = 2
***** Running Evaluation *****
  Num examples = 17
  Batch size = 2
***** Running Evaluation *****
  Num examples = 17
  Batch size = 2
***** Running Evaluation *****
  Num examples = 17
  Batch size = 2
***** Running Evaluation *****
  Num examples = 17
  Batch size = 2
Saving model checkpoint to /content/drive/MyDrive/ECE1786_Project/results/checkpoint-500
Configuration saved in /content/drive/MyDrive/ECE1786_Project/results/checkpoint-500/config.json
Model weights saved in /content/drive/MyDrive/ECE1786_Project/results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/ECE1786_Project/results/checkpoint-500/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/ECE1786_Project/results/checkpoint-500/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 17
  Batch size = 2
*****

TrainOutput(global_step=1460, training_loss=0.8454144876297206, metrics={'train_runtime': 275.5327, 'train_samples_per_second': 10.525, 'train_steps_per_second': 5.299, 'total_flos': 144865373184000.0, 'train_loss': 0.8454144876297206, 'epoch': 20.0})

In [25]:
trainer.save_model("codet5_base_finetuned_on_sorting_algorithms")

Saving model checkpoint to codet5_base_finetuned_on_sorting_algorithms
Configuration saved in codet5_base_finetuned_on_sorting_algorithms/config.json
Model weights saved in codet5_base_finetuned_on_sorting_algorithms/pytorch_model.bin
tokenizer config file saved in codet5_base_finetuned_on_sorting_algorithms/tokenizer_config.json
Special tokens file saved in codet5_base_finetuned_on_sorting_algorithms/special_tokens_map.json


In [26]:
model = AutoModelWithLMHead.from_pretrained("codet5_base_finetuned_on_sorting_algorithms")

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:1136: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
loading configuration file codet5_base_finetuned_on_sorting_algorithms/config.json
Model config T5Config {
  "_name_or_path": "codet5_base_finetuned_on_sorting_algorithms",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "bos_token_id": 1,
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 2,
  "feed_forward_proj": "relu",
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "label2id": {
    "LABEL_0": 0
 

### Evaluation

In [27]:
import random

In [28]:
random.seed(0)
random_idx = random.choice(list(finetuning_test.index))
sample_input = finetuning_test["text"][random_idx]
true_label = finetuning_test["code"][random_idx]
print('sample_input:', sample_input, '\n')
print('true_label (True Code):\n', true_label, '\n')
input_ids = tokenizer(sample_input, return_tensors='pt').input_ids
outputs = model.generate(input_ids, max_new_tokens=300)
print('generated output:\n', tokenizer.decode(outputs[0], skip_special_tokens=True))

sample_input:  Write a python program to sort an array of strings using Selection Sort
 

true_label (True Code):
 def Selection(A,n):

    for i in range(n):
        min_index = i
        min_str = A[i]
         
        for j in range(i+1,n):
            if min_str>A[j]:
                min_str = A[j]
                min_index = j
                 
        if min_index != i:
             
            temp = A[i]
            A[i] = A[min_index]
            A[min_index] = temp
 
    return A

arr = ["GeeksforGeeks", "Practice.GeeksforGeeks", "GeeksQuiz"]
 
print("Given array is")
for j in range(len(arr)):
    print(j,":",arr[j])
 
print("\nSorted array is")
for k in range(len(Selection(arr,len(arr)))):
    print(k,":",Selection(arr,len(arr))[k])



 

generated output:
 def selectionSort(arr, n):
    for i in range(1, n-1):
        min_idx = i
        for idx in range(0, n-1):
            if arr[idx] < arr[min_idx]:
                min_idx = idx
        arr[i], arr[min_idx] = arr[min_i

In [29]:
random.seed(10)
random_idx = random.choice(list(finetuning_test.index))
sample_input = finetuning_test["text"][random_idx]
true_label = finetuning_test["code"][random_idx]
print('sample_input:', sample_input, '\n')
print('true_label (True Code):\n', true_label, '\n')
input_ids = tokenizer(sample_input, return_tensors='pt').input_ids
outputs = model.generate(input_ids, max_new_tokens=300)
print('generated output:\n', tokenizer.decode(outputs[0], skip_special_tokens=True))

sample_input: write a python program to find the third distinct maximum number in this array using selection sort
 

true_label (True Code):
 def selectionSort(array):
    for i in range(len(array)-1):
        min_idx = i
        for idx in range(i + 1, len(array)):
            if array[idx] < array[min_idx]:
                min_idx = idx
        array[i], array[min_idx] = array[min_idx], array[i]
    return array
arr = [2,2,3,1]
arr = selectionSort(arr)
count = 1
prev = arr[0]
for i in range(len(arr)):
    if arr[i] != prev:
        count += 1
        prev = arr[i]
        if count == 3:
            print(arr[i])




 

generated output:
 def selectionSort(array):
    for i in range(len(array)-1):
        min_idx = i
        for idx in range(i + 1, len(array)):
            if array[idx] < array[min_idx]:
                min_idx = idx
        array[i], array[min_idx] = array[min_idx], array[i]
    return array
arr = [2,2,3,1]
arr = selectionSort(arr)
for i in range(len(arr)-1):
    if 